## Fama e Bliss (1987)

Usando os dados das LTNs

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

Para o caso brasileiro, a base de dados que será usada são os preços da Letra do Tesouro Nacional (LTN), cujo pagamento de R\$ 1000 é feito após $t$ períodos. As LTNs farão o papel dos títulos americanos.

Os dados das LTNs estão em periodicidade mensal (último dia do mês), e abrangem todo o período a partir de Dezembro de 2011 até Outubro de 2022. Essa base de dados também foi obtida via Reuters com acesso concedido pelo departamento de Economia da PUC-Rio.

In [2]:
# lendo base de dados dos retornos das LTNs
ltn_retorno = pd.read_csv('LTN_retornos.csv', sep=',', index_col=0)

In [3]:
# lendo o dicionário com o significado de cada variável do dataframe ltn_retorno
ltn_dict = pd.read_csv('LTN_dict.csv', sep=',', index_col=0)
ltn_dict

,RIC,Curve Type,Country / Region,Currency,Tenor,Constituents,Rate Type,Curve / Constituents,Contributor
Name,,,,,,,,,
Brazilian Real Zero Coupon Tomorrow Next Point,BRLPRETNZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,Tomorrow Next,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 1 Week Point,BRLPRE1WZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,1 Week,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 1 Month Point,BRLPRE1MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,1 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 2 Month Point,BRLPRE2MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,2 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 3 Month Point,BRLPRE3MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,3 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 6 Month Point,BRLPRE6MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,6 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 9 Month Point,BRLPRE9MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,9 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 1 Year Point,BRLPRE1YZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,1 Year,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 1 Year 3 Month Point,BRLPRE1Y3MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,15 Month,NaN,Spot,Constituent,REFINITIV


In [4]:
# função que transforma a coluna de datas em datas de fato
ltn_retorno.index = pd.to_datetime(ltn_retorno.index)

In [5]:
ltn_retorno

,BRLPRETNZ=R,BRLPRE1WZ=R,BRLPRE1MZ=R,BRLPRE2MZ=R,BRLPRE3MZ=R,BRLPRE6MZ=R,BRLPRE9MZ=R,BRLPRE1YZ=R,BRLPRE1Y3MZ=R,BRLPRE1Y6MZ=R,...,BRLPRE2Y9MZ=R,BRLPRE3YZ=R,BRLPRE3Y3MZ=R,BRLPRE3Y6MZ=R,BRLPRE3Y9MZ=R,BRLPRE4YZ=R,BRLPRE4Y3MZ=R,BRLPRE4Y6MZ=R,BRLPRE4Y9MZ=R,BRLPRE5YZ=R
2022-10-31,13.6503,13.6520,13.6600,13.6680,13.6830,13.6700,13.4770,13.1330,12.7362,12.3708,...,11.4985,11.4820,11.4756,11.4743,11.4767,11.4810,11.4863,11.4934,11.5040,11.5200
2022-09-30,13.6500,13.6524,13.6680,13.6820,13.6890,13.6880,13.5580,13.2450,12.9344,12.5840,...,11.6156,11.6120,11.6210,11.6353,11.6523,11.6690,11.6832,11.6963,11.7113,11.7310
2022-08-31,13.6507,13.6551,13.6730,13.7080,13.7430,13.7600,13.7380,13.5260,13.2296,12.9223,...,11.9620,11.9130,11.8925,11.8902,11.8956,11.8980,11.8897,11.8767,11.8664,11.8690
2022-07-31,13.1500,13.2074,13.5130,13.6190,13.7240,13.8790,13.9150,13.8230,13.6423,13.4427,...,12.7569,12.7140,12.6944,12.6934,12.7082,12.7370,12.7742,12.8140,12.8460,12.8630
2022-06-30,13.1450,13.1403,13.1440,13.3300,13.4790,13.7820,13.9230,13.9230,13.7418,13.5622,...,12.8624,12.7810,12.7203,12.6843,12.6764,12.6990,12.7526,12.8201,12.8793,12.9120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-04-30,9.1205,9.2502,9.3323,8.8829,8.8447,8.5166,8.3520,8.3265,8.4350,8.5899,...,9.4286,9.5563,9.6779,9.7836,9.8726,9.9437,10.0015,10.0507,10.0985,10.1433
2012-03-31,9.9405,9.7844,9.4422,9.4671,9.2289,9.1058,9.0122,8.9508,9.0529,9.2298,...,10.1583,10.2795,10.3822,10.4712,10.5436,10.6076,10.6677,10.7281,10.7935,10.8589
2012-02-29,10.8254,10.7082,10.2655,9.9632,10.1212,9.6860,9.3760,9.2789,9.3640,9.5152,...,10.2187,10.3283,10.4541,10.5755,10.6830,10.7707,10.8357,10.8856,10.9304,10.9725
2012-01-31,10.8586,10.6552,10.2868,10.2309,10.0195,9.8701,9.6671,9.5462,9.5637,9.6989,...,10.4420,10.5596,10.6588,10.7555,10.8365,10.8963,10.9347,10.9611,10.9846,11.0131


In [6]:
# dropando as variáveis que não serão utilizadas
ltn_retorno = ltn_retorno.drop(['BRLPRETNZ=R', 
                                'BRLPRE1WZ=R',
                                'BRLPRE1MZ=R', 
                                'BRLPRE2MZ=R', 
                                'BRLPRE3MZ=R', 
                                'BRLPRE6MZ=R', 
                                'BRLPRE9MZ=R', 
                                'BRLPRE1Y3MZ=R', 
                                'BRLPRE1Y6MZ=R', 
                                'BRLPRE1Y9MZ=R', 
                                'BRLPRE2Y3MZ=R', 
                                'BRLPRE2Y6MZ=R', 
                                'BRLPRE2Y9MZ=R', 
                                'BRLPRE3Y3MZ=R', 
                                'BRLPRE3Y6MZ=R', 
                                'BRLPRE3Y9MZ=R', 
                                'BRLPRE4Y3MZ=R', 
                                'BRLPRE4Y6MZ=R', 
                                'BRLPRE4Y9MZ=R'], axis=1)

Para ficar de acordo com o paper de Fama e Bliss (1987), divide-se o valor das LTNs por 1000 de forma que o pagamento agora, é de R\$1.

In [7]:
# normalizando os retornos das LTNs para decimal
ltn_retorno = ltn_retorno/100

In [8]:
ltn_retorno

,BRLPRE1YZ=R,BRLPRE2YZ=R,BRLPRE3YZ=R,BRLPRE4YZ=R,BRLPRE5YZ=R
2022-10-31,0.131330,0.117980,0.114820,0.114810,0.115200
2022-09-30,0.132450,0.119380,0.116120,0.116690,0.117310
2022-08-31,0.135260,0.123750,0.119130,0.118980,0.118690
2022-07-31,0.138230,0.130640,0.127140,0.127370,0.128630
2022-06-30,0.139230,0.132310,0.127810,0.126990,0.129120
...,...,...,...,...,...
2012-04-30,0.083265,0.089309,0.095563,0.099437,0.101433
2012-03-31,0.089508,0.096231,0.102795,0.106076,0.108589
2012-02-29,0.092789,0.098339,0.103283,0.107707,0.109725
2012-01-31,0.095462,0.100375,0.105596,0.108963,0.110131


Como $BRLPRExYZ=R$ é o rendimento anualizado de uma LTN comprada em $t$ que paga R\$1 em $t+x$, enquanto $r(x:t)$ é o rendimento não anualizado de uma LTN comprada em $t$ que paga R\$1 em $t$. Portanto

$$
r(x:t) = (1+BRLPRE(x)YZ=R)^x -1
$$

In [9]:
# criando as taxas de retornos das LTNs
ltn_retorno['r(1:t)'] = ltn_retorno['BRLPRE1YZ=R']
ltn_retorno['r(2:t)'] = (1+ltn_retorno['BRLPRE2YZ=R'])**2 - 1
ltn_retorno['r(3:t)'] = (1+ltn_retorno['BRLPRE3YZ=R'])**3 - 1
ltn_retorno['r(4:t)'] = (1+ltn_retorno['BRLPRE4YZ=R'])**4 - 1
ltn_retorno['r(5:t)'] = (1+ltn_retorno['BRLPRE5YZ=R'])**5 - 1

No artigo Fama e Bliss (1987), definem o rendimento de um título que paga \$1 em $t+x$ comprado em $t$ como 

$$
r(x:t) = \ln\left[\frac{p(x:t+x)}{p(x:t)}\right]
$$

Como $p(x:t+x) = 1$

$$
r(x:t) = -\ln[{p(x:t)}]
$$

Portanto, somos capazes de recuperar os preços das LTNs, embora não faça sentido uma vez que manipularíamos os preços para encontrar as taxas.

Uma outra taxa que será necessária é $r(x:t+1)$, interpretada como o retorno de uma LTN comprada em $t+1$ que paga R\$1 em $t+x+1$. A equação que nos entrega essa taxa é dada por

$$
r(x:t+1) = \frac{[1+r(x-1:t)]}{[1+r(1:t)]} - 1
$$

In [10]:
ltn_retorno['r(1:t+1)'] = ((1+ltn_retorno['r(2:t)'])/(1+ltn_retorno['r(1:t)'])) - 1
ltn_retorno['r(2:t+1)'] = ((1+ltn_retorno['r(3:t)'])/(1+ltn_retorno['r(1:t)'])) - 1
ltn_retorno['r(3:t+1)'] = ((1+ltn_retorno['r(4:t)'])/(1+ltn_retorno['r(1:t)'])) - 1
ltn_retorno['r(4:t+1)'] = ((1+ltn_retorno['r(5:t)'])/(1+ltn_retorno['r(1:t)'])) - 1

A última taxa necessária para a construção das variáveis é $r(1:t+x)$ que representa o retorno da LTN comprada em $t+x$ que paga R\$1 em $t+x+1$ obtida através de

$$
r(1:t+x) = \frac{[1+r(x+1:t)]}{[1+r(x:t)]} -1
$$

In [11]:
ltn_retorno['r(1:t+2)'] = ((1+ltn_retorno['r(3:t)'])/(1+ltn_retorno['r(2:t)'])) - 1
ltn_retorno['r(1:t+3)'] = ((1+ltn_retorno['r(4:t)'])/(1+ltn_retorno['r(3:t)'])) - 1
ltn_retorno['r(1:t+4)'] = ((1+ltn_retorno['r(5:t)'])/(1+ltn_retorno['r(4:t)'])) - 1

In [12]:
ltn_retorno

,BRLPRE1YZ=R,BRLPRE2YZ=R,BRLPRE3YZ=R,BRLPRE4YZ=R,BRLPRE5YZ=R,r(1:t),r(2:t),r(3:t),r(4:t),r(5:t),r(1:t+1),r(2:t+1),r(3:t+1),r(4:t+1),r(1:t+2),r(1:t+3),r(1:t+4)
2022-10-31,0.131330,0.117980,0.114820,0.114810,0.115200,0.131330,0.249879,0.385525,0.544555,0.724900,0.104788,0.224687,0.365256,0.524665,0.108527,0.114780,0.116761
2022-09-30,0.132450,0.119380,0.116120,0.116690,0.117310,0.132450,0.253012,0.390377,0.555000,0.741279,0.106461,0.227760,0.373129,0.537621,0.109628,0.118402,0.119793
2022-08-31,0.135260,0.123750,0.119130,0.118980,0.118690,0.135260,0.262814,0.401657,0.567795,0.752059,0.112357,0.234657,0.381001,0.543311,0.109947,0.118530,0.117531
2022-07-31,0.138230,0.130640,0.127140,0.127370,0.128630,0.138230,0.278347,0.431969,0.615347,0.831293,0.123101,0.258066,0.419175,0.608896,0.120172,0.128060,0.133684
2022-06-30,0.139230,0.132310,0.127810,0.126990,0.129120,0.139230,0.282126,0.434524,0.613170,0.835272,0.125432,0.259205,0.416018,0.610976,0.118864,0.124534,0.137680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-04-30,0.083265,0.089309,0.095563,0.099437,0.101433,0.083265,0.186594,0.314959,0.461105,0.621028,0.095387,0.213884,0.348797,0.496428,0.108179,0.111141,0.109453
2012-03-31,0.089508,0.096231,0.102795,0.106076,0.108589,0.089508,0.201722,0.341172,0.496718,0.674375,0.102995,0.230988,0.373756,0.536818,0.116041,0.115978,0.118698
2012-02-29,0.092789,0.098339,0.103283,0.107707,0.109725,0.092789,0.206349,0.342953,0.505565,0.682972,0.103917,0.228922,0.377727,0.540070,0.113238,0.121086,0.117834
2012-01-31,0.095462,0.100375,0.105596,0.108963,0.110131,0.095462,0.210825,0.351417,0.512405,0.686053,0.105310,0.233650,0.380610,0.539125,0.116112,0.119126,0.114815


A taxa forward é definida como

$$
f(x,x-1:t) = r(x:t) - r(x-1:t)
$$

Enquanto a taxa de retorno no caso brasileiro, da LTN comprada em $t$ e vendida em $t+1$ que pagaria R\$1 em $t+x-1$ é dada por

$$
h(x,x-1:t+1) = r(x:t) - r(x-1:t+1)
$$

In [13]:
ltn_retorno['f(2,1:t)'] = ltn_retorno['r(2:t)'] - ltn_retorno['r(1:t)']
ltn_retorno['f(3,2:t)'] = ltn_retorno['r(3:t)'] - ltn_retorno['r(2:t)']
ltn_retorno['f(4,3:t)'] = ltn_retorno['r(4:t)'] - ltn_retorno['r(3:t)']
ltn_retorno['f(5,4:t)'] = ltn_retorno['r(5:t)'] - ltn_retorno['r(4:t)']


ltn_retorno['h(2,1:t+1)'] = ltn_retorno['r(2:t)'] - ltn_retorno['r(1:t+1)']
ltn_retorno['h(3,2:t+1)'] = ltn_retorno['r(3:t)'] - ltn_retorno['r(2:t+1)']
ltn_retorno['h(4,3:t+1)'] = ltn_retorno['r(4:t)'] - ltn_retorno['r(3:t+1)']
ltn_retorno['h(5,4:t+1)'] = ltn_retorno['r(5:t)'] - ltn_retorno['r(4:t+1)']

A regressão da Tabela 1 é dada por

$$
h(x,x-1:t+1) - r(1:t) = \alpha + \beta \left[f(x,x-1:t) - r(1:t)\right] + u_2(t+1)
$$

onde o regressor é $f(x,x-1:t) - r(1:t)$ e o regressando $h(x,x-1:t+1) - r(1:t)$

In [14]:
ltn_retorno['y'] = ltn_retorno['h(2,1:t+1)'] - ltn_retorno['r(1:t)']
ltn_retorno['x'] = ltn_retorno['f(2,1:t)'] - ltn_retorno['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $x=2$, a regressão é


$$
h(2,1:t+1) - r(1:t) = \alpha + \beta \left[f(2,1:t) - r(1:t)\right] + u_2(t+1)
$$

In [15]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     4.905
Date:                Fri, 21 Oct 2022   Prob (F-statistic):             0.0285
Time:                        19:41:12   Log-Likelihood:                 488.15
No. Observations:                 131   AIC:                            -972.3
Df Residuals:                     129   BIC:                            -966.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0119      0.001     12.203      0.0

In [16]:
ltn_retorno['y'] = ltn_retorno['h(3,2:t+1)'] - ltn_retorno['r(1:t)']
ltn_retorno['x'] = ltn_retorno['f(3,2:t)'] - ltn_retorno['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $x=3$, a regressão é


$$
h(3,2:t+1) - r(1:t) = \alpha + \beta \left[f(3,2:t) - r(1:t)\right] + u_2(t+1)
$$

In [17]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.461
Date:                Fri, 21 Oct 2022   Prob (F-statistic):              0.229
Time:                        19:41:12   Log-Likelihood:                 388.96
No. Observations:                 131   AIC:                            -773.9
Df Residuals:                     129   BIC:                            -768.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0247      0.003      9.094      0.0

In [18]:
ltn_retorno['y'] = ltn_retorno['h(4,3:t+1)'] - ltn_retorno['r(1:t)']
ltn_retorno['x'] = ltn_retorno['f(4,3:t)'] - ltn_retorno['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $x=4$, a regressão é


$$
h(4,3:t+1) - r(1:t) = \alpha + \beta \left[f(4,3:t) - r(1:t)\right] + u_2(t+1)
$$

In [19]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.591
Date:                Fri, 21 Oct 2022   Prob (F-statistic):              0.210
Time:                        19:41:13   Log-Likelihood:                 327.80
No. Observations:                 131   AIC:                            -651.6
Df Residuals:                     129   BIC:                            -645.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0283      0.006      5.039      0.0

In [20]:
ltn_retorno['y'] = ltn_retorno['h(5,4:t+1)'] - ltn_retorno['r(1:t)']
ltn_retorno['x'] = ltn_retorno['f(5,4:t)'] - ltn_retorno['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $x=5$, a regressão é


$$
h(5,4:t+1) - r(1:t) = \alpha + \beta \left[f(5,4:t) - r(1:t)\right] + u_2(t+1)
$$

In [21]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.127
Method:                 Least Squares   F-statistic:                     19.90
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.76e-05
Time:                        19:41:13   Log-Likelihood:                 289.81
No. Observations:                 131   AIC:                            -575.6
Df Residuals:                     129   BIC:                            -569.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0147      0.008      1.791      0.0

Para replicar o primeiro modelo da Tabela 3, basta alterar a variável $h(x,x-1:t+1)$ por $r(1:t+x-1)$, isso é

$$
r(1:t+x-1) - r(1:t) = \alpha + \beta \left[f(x,x-1:t) - r(1:t)\right] + u_2(t+1)
$$

In [22]:
ltn_retorno['y'] = ltn_retorno['r(1:t+1)'] - ltn_retorno['r(1:t)']
ltn_retorno['x'] = ltn_retorno['f(2,1:t)'] - ltn_retorno['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $x=2$, a regressão é


$$
r(1:t+1) - r(1:t) = \alpha + \beta \left[f(2,1:t) - r(1:t)\right] + u_2(t+1)
$$

In [23]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.827
Method:                 Least Squares   F-statistic:                     624.3
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           2.84e-51
Time:                        19:41:13   Log-Likelihood:                 488.15
No. Observations:                 131   AIC:                            -972.3
Df Residuals:                     129   BIC:                            -966.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0119      0.001    -12.203      0.0

In [24]:
ltn_retorno['y'] = ltn_retorno['r(1:t+2)'] - ltn_retorno['r(1:t)']
ltn_retorno['x'] = ltn_retorno['f(3,2:t)'] - ltn_retorno['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $x=3$, a regressão é


$$
r(1:t+2) - r(1:t) = \alpha + \beta \left[f(3,2:t) - r(1:t)\right] + u_2(t+1)
$$

In [25]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.668
Model:                            OLS   Adj. R-squared:                  0.666
Method:                 Least Squares   F-statistic:                     259.8
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.06e-32
Time:                        19:41:14   Log-Likelihood:                 396.31
No. Observations:                 131   AIC:                            -788.6
Df Residuals:                     129   BIC:                            -782.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0221      0.003     -8.586      0.0

In [26]:
ltn_retorno['y'] = ltn_retorno['r(1:t+3)'] - ltn_retorno['r(1:t)']
ltn_retorno['x'] = ltn_retorno['f(4,3:t)'] - ltn_retorno['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $x=4$, a regressão é


$$
r(1:t+3) - r(1:t) = \alpha + \beta \left[f(4,3:t) - r(1:t)\right] + u_2(t+1)
$$

In [27]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.374
Method:                 Least Squares   F-statistic:                     78.60
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           5.28e-15
Time:                        19:41:14   Log-Likelihood:                 346.85
No. Observations:                 131   AIC:                            -689.7
Df Residuals:                     129   BIC:                            -683.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0219      0.005     -4.510      0.0

In [28]:
ltn_retorno['y'] = ltn_retorno['r(1:t+4)'] - ltn_retorno['r(1:t)']
ltn_retorno['x'] = ltn_retorno['f(5,4:t)'] - ltn_retorno['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $x=5$, a regressão é


$$
r(1:t+4) - r(1:t) = \alpha + \beta \left[f(5,4:t) - r(1:t)\right] + u_2(t+1)
$$

In [29]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     19.81
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.83e-05
Time:                        19:41:15   Log-Likelihood:                 322.25
No. Observations:                 131   AIC:                            -640.5
Df Residuals:                     129   BIC:                            -634.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0080      0.006     -1.251      0.2